In [1]:
# %load rgcn.py
import argparse
import os.path as osp
import time

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv
from torch_geometric.utils import k_hop_subgraph
'''
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='AIFB',
                    choices=['AIFB', 'MUTAG', 'BGS', 'AM'])
args = parser.parse_args()

# Trade memory consumption for faster computation.
if args.dataset in ['AIFB', 'MUTAG']:
    Conv = FastRGCNConv
else:
    Conv = RGCNConv
'''
Conv = FastRGCNConv
path = './'#osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Entities')
#dataset = Entities(path, args.dataset)
dataset = Entities(path, 'AIFB')
data = dataset[0]

# BGS and AM graphs are too big to process them in a full-batch fashion.
# Since our model does only make use of a rather small receptive field, we
# filter the graph to only contain the nodes that are at most 2-hop neighbors
# away from any training/test node.
node_idx = torch.cat([data.train_idx, data.test_idx], dim=0)
node_idx, edge_index, mapping, edge_mask = k_hop_subgraph(
    node_idx, 2, data.edge_index, relabel_nodes=True)

data.num_nodes = node_idx.size(0)
data.edge_index = edge_index
data.edge_type = data.edge_type[edge_mask]
data.train_idx = mapping[:data.train_idx.size(0)]
data.test_idx = mapping[data.train_idx.size(0):]


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv(data.num_nodes, 16, dataset.num_relations,
                          num_bases=30)
        self.conv2 = Conv(16, dataset.num_classes, dataset.num_relations,
                          num_bases=30)

    def forward(self, edge_index, edge_type):
        x = F.relu(self.conv1(None, edge_index, edge_type))
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)


if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

device = torch.device('cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = float((pred[data.train_idx] == data.train_y).float().mean())
    test_acc = float((pred[data.test_idx] == data.test_y).float().mean())
    return train_acc, test_acc


times = []
for epoch in range(1, 51):
    start = time.time()
    loss = train()
    train_acc, test_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f} '
          f'Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")


Extracting .\aifb.tgz
Processing...
Done!


Epoch: 01, Loss: 1.4094, Train: 0.9357 Test: 0.8611
Epoch: 02, Loss: 0.7527, Train: 0.9429 Test: 0.8611
Epoch: 03, Loss: 0.3182, Train: 0.9429 Test: 0.9167
Epoch: 04, Loss: 0.1464, Train: 0.9714 Test: 0.9167
Epoch: 05, Loss: 0.0923, Train: 0.9786 Test: 0.9167
Epoch: 06, Loss: 0.0619, Train: 0.9857 Test: 0.9167
Epoch: 07, Loss: 0.0399, Train: 0.9857 Test: 0.9444
Epoch: 08, Loss: 0.0241, Train: 0.9929 Test: 0.9444
Epoch: 09, Loss: 0.0138, Train: 1.0000 Test: 0.9444
Epoch: 10, Loss: 0.0079, Train: 1.0000 Test: 0.9444
Epoch: 11, Loss: 0.0047, Train: 1.0000 Test: 0.9444
Epoch: 12, Loss: 0.0031, Train: 1.0000 Test: 0.9444
Epoch: 13, Loss: 0.0022, Train: 1.0000 Test: 0.9444
Epoch: 14, Loss: 0.0017, Train: 1.0000 Test: 0.9444
Epoch: 15, Loss: 0.0013, Train: 1.0000 Test: 0.9444
Epoch: 16, Loss: 0.0011, Train: 1.0000 Test: 0.9444
Epoch: 17, Loss: 0.0009, Train: 1.0000 Test: 0.9167
Epoch: 18, Loss: 0.0008, Train: 1.0000 Test: 0.9167
Epoch: 19, Loss: 0.0008, Train: 1.0000 Test: 0.9167
Epoch: 20, L